In [1]:
liked_books = ["4408", "31147619", "29983711", "9401317", "9317691", "8153988", "20494944"]

In [5]:
with open("book_id_map.csv") as f:
    for _ in range(10):
        print(f.readline())

book_id_csv,book_id

0,34684622

1,34536488

2,34017076

3,71730

4,30422361

5,33503613

6,33517540

7,34467031

8,6383669



In [7]:
csv_book_mapping = {}
with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [10]:
csv_book_mapping['0']

'34684622'

In [ ]:
overlap_users = set()
with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue


        try:
            rating = int(rating)
        except ValueError:
            continue
        
        book_id = csv_book_mapping[csv_id]
        
        if book_id in liked_books and rating >= 4:
                overlap_users.add(user_id)
rec_lines = []

In [ ]:
with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = csv_book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

In [ ]:
import pandas as pd


recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [ ]:
top_recs = recs["book_id"].value_counts().head(10)
top_recs = top_recs.index.values

In [ ]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)
books_titles.head()

In [ ]:
all_recs = recs["book_id"].value_counts()
all_recs = all_recs.to_frame().reset_index()
all_recs.columns = ["book_id", "book_count"]
all_recs.head(5)

In [ ]:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id")
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["ratings"])
all_recs.sort_values("score", ascending=False).head(10)

In [ ]:
popular_recs = all_recs[all_recs["book_count"] > 200].sort_values("score", ascending=False)

In [ ]:
def make_clickable(val):
    return '{}">Goodreads'.format(val, val)


def show_image(val):
    return '{}">{}" width=50>'.format(val, val)




popular_recs[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})